In [12]:
from kafka import KafkaConsumer
from time import sleep
from json import dumps,loads
import json
from s3fs import S3FileSystem

In [13]:
# Setup Kafka consumer
consumer = KafkaConsumer(
    'testing2',
    bootstrap_servers=['{your-host}:9092'], # Replace with your Kafka broker IP
    auto_offset_reset='earliest', # Read the message from the beginning
    enable_auto_commit=True,
    group_id='my-group', # Make sure to use the appropriate group_id
    value_deserializer=lambda x: loads(x.decode('utf-8'))
)

data = []

#max_messages = 100 # Limit the number of messages to receive

#for _ in range(max_messages):
#    message = next(consumer)
#    print(f"Received message: {message.value}") # Print the received data to confirm
#    data.append(message.value)

In [14]:
max_messages = 100 # Limit the number of messages to receive
message_count = 0 # Counter of the number of messages received

for message in consumer:
    print(f"Received message: {message.value}") # Print the received data to confirm
    data.append(message.value)
    message_count += 1
    if message_count >= max_messages:
        break

Received message: {'Datetime': '2024-06-17 09:30:00-04:00', 'Adj Close_AAPL': 212.9936065673828, 'Adj Close_IBM': 168.1300048828125, 'Adj Close_TSLA': 180.19970703125, 'Close_AAPL': 212.9936065673828, 'Close_IBM': 168.1300048828125, 'Close_TSLA': 180.19970703125, 'High_AAPL': 214.44000244140625, 'High_IBM': 168.75999450683594, 'High_TSLA': 180.3000030517578, 'Low_AAPL': 212.9600067138672, 'Low_IBM': 167.82000732421875, 'Low_TSLA': 177.0, 'Open_AAPL': 213.3000030517578, 'Open_IBM': 168.75999450683594, 'Open_TSLA': 177.9499969482422, 'Volume_AAPL': 5198111, 'Volume_IBM': 139002.0, 'Volume_TSLA': 5019092}
Received message: {'Datetime': '2024-06-17 09:35:00-04:00', 'Adj Close_AAPL': 213.3159942626953, 'Adj Close_IBM': 167.5449981689453, 'Adj Close_TSLA': 180.16720581054688, 'Close_AAPL': 213.3159942626953, 'Close_IBM': 167.5449981689453, 'Close_TSLA': 180.16720581054688, 'High_AAPL': 213.7050018310547, 'High_IBM': 168.25, 'High_TSLA': 180.94000244140625, 'Low_AAPL': 212.72000122070312, 'Lo

In [15]:
# Create a DataFrame from the received data
if data:
    df = pd.DataFrame(data)
    print(df.head()) # Print the data to ensure the DataFrame is initialized correctly
else:
    print("No data received from Kafka.")

                    Datetime  Adj Close_AAPL  Adj Close_IBM  Adj Close_TSLA  \
0  2024-06-17 09:30:00-04:00      212.993607     168.130005      180.199707   
1  2024-06-17 09:35:00-04:00      213.315994     167.544998      180.167206   
2  2024-06-17 09:40:00-04:00      213.723999     168.020004      180.408005   
3  2024-06-17 09:45:00-04:00      213.570007     168.460007      180.869995   
4  2024-06-17 09:50:00-04:00      213.463104     168.300003      181.065002   

   Close_AAPL   Close_IBM  Close_TSLA   High_AAPL    High_IBM   High_TSLA  \
0  212.993607  168.130005  180.199707  214.440002  168.759995  180.300003   
1  213.315994  167.544998  180.167206  213.705002  168.250000  180.940002   
2  213.723999  168.020004  180.408005  214.080002  168.080002  180.739899   
3  213.570007  168.460007  180.869995  214.800003  168.460007  181.610001   
4  213.463104  168.300003  181.065002  213.589996  168.699997  181.490005   

     Low_AAPL     Low_IBM    Low_TSLA   Open_AAPL    Open_IBM 

In [16]:
print(df.columns)

Index(['Datetime', 'Adj Close_AAPL', 'Adj Close_IBM', 'Adj Close_TSLA',
       'Close_AAPL', 'Close_IBM', 'Close_TSLA', 'High_AAPL', 'High_IBM',
       'High_TSLA', 'Low_AAPL', 'Low_IBM', 'Low_TSLA', 'Open_AAPL', 'Open_IBM',
       'Open_TSLA', 'Volume_AAPL', 'Volume_IBM', 'Volume_TSLA'],
      dtype='object')


In [17]:
df.head(5)

,Datetime,Adj Close_AAPL,Adj Close_IBM,Adj Close_TSLA,Close_AAPL,Close_IBM,Close_TSLA,High_AAPL,High_IBM,High_TSLA,Low_AAPL,Low_IBM,Low_TSLA,Open_AAPL,Open_IBM,Open_TSLA,Volume_AAPL,Volume_IBM,Volume_TSLA
0,2024-06-17 09:30:00-04:00,212.993607,168.130005,180.199707,212.993607,168.130005,180.199707,214.440002,168.759995,180.300003,212.960007,167.820007,177.000000,213.300003,168.759995,177.949997,5198111,139002.0,5019092
1,2024-06-17 09:35:00-04:00,213.315994,167.544998,180.167206,213.315994,167.544998,180.167206,213.705002,168.250000,180.940002,212.720001,167.500000,179.699997,212.994995,168.154999,180.195007,1672453,41521.0,2596640
2,2024-06-17 09:40:00-04:00,213.723999,168.020004,180.408005,213.723999,168.020004,180.408005,214.080002,168.080002,180.739899,213.250000,167.500000,179.419998,213.330002,167.535004,180.179993,1409081,35748.0,2257603
3,2024-06-17 09:45:00-04:00,213.570007,168.460007,180.869995,213.570007,168.460007,180.869995,214.800003,168.460007,181.610001,213.229996,168.000900,180.414993,213.720001,168.020004,180.419998,2134755,30790.0,2477614
4,2024-06-17 09:50:00-04:00,213.463104,168.300003,181.065002,213.463104,168.300003,181.065002,213.589996,168.699997,181.490005,212.960007,168.300003,180.440002,213.559998,168.520004,180.860001,1350967,20405.0,1419959


In [ ]:
#import time

#time_limit = 10 # Time limit in seconds
#start_time = time.time()

#while time.time() - start_time < time_limit:
#    message = next(consumer)
#    print(f"Received message: {message.value}") # Print the received data to confirm
#    data.append(message.value)

In [ ]:
#for message in consumer:
    #print(f"Received message: {message.value}")

In [18]:
df.to_csv('received_data.csv', index=False)

In [ ]:
#for c in consumer:
#    print(c.value)

In [23]:
# Setup PostgreSQL connection
import psycopg2

conn = psycopg2.connect(
    dbname='{your-db-name}',
    user='{your-user-name}',
    password='{your-pass-name}',
    host='{your-host}',
    port='{your-port}'
)
cursor = conn.cursor()

In [ ]:
# Create table if not exists
cursor.execute("""
    CREATE TABLE IF NOT EXISTS new_datastocks (
        datetime TIMESTAMP,
        adj_close_aapl FLOAT,
        adj_close_ibm FLOAT,
        adj_close_tsla FLOAT,
        close_aapl FLOAT,
        close_ibm FLOAT,
        close_tsla FLOAT,
        high_aapl FLOAT,
        high_ibm FLOAT,
        high_tsla FLOAT,
        low_aapl FLOAT,
        low_ibm FLOAT,
        low_tsla FLOAT,
        open_aapl FLOAT,
        open_ibm FLOAT,
        open_tsla FLOAT,
        volume_aapl FLOAT,
        volume_ibm FLOAT,
        volume_tsla FLOAT
    );
""")
conn.commit()

In [24]:
# Insert data into PostgreSQL
if data:
    for record in data:
        cursor.execute(
            """
            INSERT INTO new_datastocks (datetime, adj_close_aapl, adj_close_ibm, adj_close_tsla, close_aapl, close_ibm, close_tsla, 
                                    high_aapl, high_ibm, high_tsla, low_aapl, low_ibm, low_tsla, open_aapl, open_ibm, open_tsla, 
                                    volume_aapl, volume_ibm, volume_tsla) 
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """,
            (
                record['Datetime'],
                record['Adj Close_AAPL'], record['Adj Close_IBM'], record['Adj Close_TSLA'],
                record['Close_AAPL'], record['Close_IBM'], record['Close_TSLA'],
                record['High_AAPL'], record['High_IBM'], record['High_TSLA'],
                record['Low_AAPL'], record['Low_IBM'], record['Low_TSLA'],
                record['Open_AAPL'], record['Open_IBM'], record['Open_TSLA'],
                record['Volume_AAPL'], record['Volume_IBM'], record['Volume_TSLA']
            )
        )
    conn.commit()